# Imports and Settings

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.datasets import make_regression, load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE, RFECV, f_regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RandomizedLasso
from sklearn.metrics import r2_score
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from minepy import MINE

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

One of the most important things in Machine Learning is the __statement of attributes__, which involves both the part of __selection of attributes__ (_feature selection_) and __creation of attributes__ (_feature creation_). In this notebook, we will focus on the selection of attributes.

Most machine learning algorithms suffer from a problem: if you give them bad attributes, the result is likely to be bad as well. Bad attributes involve from inconsistent or wrong data to noise in the data. This is even more crucial when the number of attributes is very large. In general, you do not need to use all the attributes. In addition to reducing training time, fewer attributes also mean fewer problems to worry about. It's like that old diatado says: _"Less is more!"_

The selection of attributes is important because:

- leaves the __training faster__, shortening training time;
- __reduces the complexity of a model__ and makes it simpler to interpret;
- __facilitates the understanding of the relation of the attributes with the output__;
- improves the accuracy of the model if the "optimal" subset is chosen; and
- __reduces the chance of overfitting and improves generalization__

We can group the attribute selection algorithms into 3 types: __filter methods__, __packaging methods__ (_wrapper methods_) and __carried methods__ (_embedded methods_). In the following sections, we'll look at the differences between them.
<hr>

## Filter Methods
<img src='images/select_filter_methods.png'>

Filtering methods, also known as "univariate attribute selection" (_univariate feature selection_), are generally used as pre-processing. The selection of attributes is independent of any machine learning algorithm. Instead, each attribute is examined individually to determine the strength of its relationship to the variable response (output). These methods are simple to run, understand, and are generally good at getting a good understanding of data (but not necessarily optimizing the set of attributes for better generalization). There are a lot of filtering methods. As a basic guide, you can consider the following table that defines some of the univariate selection methods:

| Atributo\Resposta |        Continuous       | Category            |
|:-----------------:|:---------------------:|:---------------------:|
|   __Continuous__    | Pearson's Correlation | LDA                   |
|  __Category__   |         ANOVA         | Chi-Square ($\chi^2$) |


- __Pearson's correlation__: used to quantify the linear correlation between two continuous variables $X$ and $Y$. The output varies from $[- 1, +1]$, where $ -1 $ means perfect negative correlation (when $ X $ grows, $ Y $ decreases) and $ + 1 $ means perfect positive correlation (when $ X $ grows , $ Y $ grows). The Pearson correlation coefficient $ P (X, Y) $ is given by:

$$ P(X,Y) = \frac{cov(X, Y)}{\sigma_X \sigma_Y} $$

- __LDA__: stands for Linear Discriminant Analysis (_linear discriminant analysis_) and is used to find the best linear combination of attributes that separate two or more classes. You can check out a full notebook on LDA in this same repository by clicking [here] (LDA.ipynb).
- __ANOVA__: statistical test that means __analysis of variance__. It is similar to LDA, but functions for one or more independent categorical attributes and a continuous attribute as output. This test indicates whether the average of several groups are equal or not.
- __Chi-square ($\chi^2$) __: is also a statistical test, only applied to groups of categorical attributes to evaluate the probability of correlation or association between them using their frequency distributions.

One thing to keep in mind is that filtering methods do NOT remove multicollinearity. Therefore, you should also be concerned with the multicollinearity of attributes before you train your model.
<hr>

## Pearson's Correlation
__One disadvantage of Pearson's correlation coefficient is that it is sensitive only to linear relations.__ If the relationship is nonlinear, the Pearson correlation can be close to zero even if there is a 1: 1 match between the two variables. For example, the correlation between $x$ and $x^2$ is approximately 0:

In [2]:
x = np.random.uniform(-1, 1, 1000)
print(np.corrcoef(x.T, x**2))

[[ 1.          0.05214981]
 [ 0.05214981  1.        ]]


You can check some examples of the Pearson coefficient for some data types:

<img src='images/selecao_pearson.png'>

In addition, we can see, in the figure below, different data that has the same number of points and the same Pearson correlation coefficient, but totally different graphs. So it's always worth plotting the data, if possible:

<img src='images/selecao_pearson_plots.png'>

## Mutual Information
A more robust option to the Pearson coefficient is the __mutual information (MI) __, which measures the mutual dependence between variables, usually in bits. Mutual information is defined as:

$$I(X, Y) = \sum_{y\in Y} \sum_{x \in X} p(x, y)log \left(\frac{p(x,y)}{p(x)p(y)}\right)$$

However, it can be inconvenient when used directly for attribute ranking for two reasons:

1. __It is not a metric and is not normalized__. Therefore, IM values can not be compared between two datasets.
2. __It is not suitable for continuous variables__. In general, the variables need to be discretized for the construction of the bins, however the information of the score can be very sensitive the selection of the bins.
<hr>

## Maximal Information Coefficient
__Maximal Information Coefficient (MIC) __ is a technique developed to address the deficiencies of _mutual information_. It looks for optimal bins and still normalizes the _mutual information_ score between $[0, 1]$. In python, it can be implemented using the __minepy__ library:

In [3]:
x = np.random.uniform(-1, 1, 1000)

m = MINE()
m.compute_score(x, x**2)
print(m.mic())

1.0000000000000002


However, there are some criticisms ([here](http://ie.technion.ac.il/~gorfinm/files/science6.pdf) e [here](http://www-stat.stanford.edu/~tibs/reshef/comment.pdf)) regarding the statistical power of the MIC, that is, the ability to reject the null hypothesis when it is false. This may or may not be a problem, depending on the bank and its noise.
<hr>

## Wrapper Methods
<img src='images/selecao_wrapper_methods.png' width="700">

Packaging methods try to use a subset of attributes and train a model with this subset. __ Based on the result of the previous model, the algorithm decides to add or remove attributes from the subset__. This problem is essentially reduced to a search problem. __The packaging methods are usually very computationally expensive__.

The most common packaging algorithms are divided into 3 groups:

- __Forward Selection__: iterative method in which the algorithm starts without any attribute in the model. At each iteration, the algorithm adds the attribute that provides the highest performance gain. The algorithm ends when adding new attributes does not improve algorithm performance.
- __Backward Elimination__: In this method, the algorithm starts with all attributes and, at each iteration, removes the attribute with the least significance or, when removed, improves the performance of the model. This procedure is repeated until no improvement is observed in the removal of the attributes.
- __Recursive Feature Elimination__: is a greedy optimization algorithm whose goal is to find the best subset of attributes. Repeatedly, the algorithm creates models and holds the best or worst attribute at each iteration. So a new model is trained with the best attributes until the attributes run out. Finally, the algorithm makes the ranking of attributes based on the order of their deletions.

Below, we'll look at some of the most commonly used packaging methods
<hr>

## Stability Selection
[Stability selection](http://stat.ethz.ch/~nicolai/stability.pdf) is considered a relatively new attribute selection method, based on the subsampling and combination of selection algorithms (which can be regressors, SVMs or other similar methods). __The main idea is to apply an attribute selector algorithm in different subsets of the data and with different subset of the attributes__. After repeating this process a number of times, the selection results can be aggregated, for example, by counting how many times an attribute was selected as important when it was in a subset of attributes. It is expected that important attributes score close to 100%, since they should always be selected when possible. Weak attributes, but still relevant, will have non-zero scores, since they were selected when important attributes were not present in the current selected subset. Unreliable attributes, in turn, should have scores (close to) zero, since they should not be present in the selected attributes.

Sklearn implements stability selection in the [randomized lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLasso.html) e [randomized logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLogisticRegression.html).

In [ ]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

rlasso = RandomizedLasso(alpha=0.025, random_state=42)
rlasso.fit(x, y)

print("Atributos ordenados pelo score:")
print(sorted(zip(rlasso.scores_, names), reverse=True))

As you can see, the 3 most important attributes have a score equal to 1.0, that is, they have always been chosen as useful attributes. Obviously, this result may be different for other throttling parameters, even though the implementation of sklearn is able to choose a good $ \ alpha $ automatically. The rest of the scores decrease dramatically after these 3 attributes, however the attributes are usually not as aggressive as in the case of pure Lasso or Random Forest. __This means that the stability selected is useful both in pure attribute selection to reduce overfitting and in data interpretation__. In general, good attributes will have 0 as a coefficient just because they are similar or correlated in the dataset (as in the Lasso regression). __For selection of attributes, stability selection is among the best algorithms tested in different databases and configurations__.
<hr>

## Recursive deletion of attributes
__Recursive feature elimination, RFE__ is based on the idea of repeatedly constructing a model (eg SVM or regressor)__and choosing the best or worst attribute__ (for example, based on the coefficients), __ separating this attribute and repeating the process with the others__. This process is applied until all the attributes in the dataset have been used. __The attributes are then ranked according to the order of elimination__. Therefore, this represents a greedy optimization process to find the best subset of attributes.

__The stability of the RFE is quite dependent on the type of model used for attribute ranking in each iteration__.

Sklearn provides the [RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) for recursive attribute deletion and [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html) to find the rankings and optimal quantity of attributes via cross-validation.

In [4]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

# uses linear regression as a model
lr = LinearRegression()
rfe = RFE(estimator=lr, n_features_to_select=1)
rfe.fit(x, y)

print("Attributes sorted rank:")
print(sorted(zip(rfe.ranking_, names)))

Attributes sorted rank:
[(1, 'NOX'), (2, 'RM'), (3, 'CHAS'), (4, 'PTRATIO'), (5, 'DIS'), (6, 'LSTAT'), (7, 'RAD'), (8, 'CRIM'), (9, 'INDUS'), (10, 'ZN'), (11, 'TAX'), (12, 'B'), (13, 'AGE')]


In [5]:
boston = load_boston()
x = boston['data']
y = boston['target']
names = boston['feature_names']

# uses linear regression as a model
lr = LinearRegression()
rfe = RFECV(estimator=lr, step=1, cv=10, n_jobs=-1)
rfe.fit(x, y)

print("Attributes sorted rank:")
print(sorted(zip(rfe.ranking_, names)))

Attributes sorted rank:
[(1, 'CHAS'), (1, 'DIS'), (1, 'LSTAT'), (1, 'NOX'), (1, 'PTRATIO'), (1, 'RM'), (2, 'RAD'), (3, 'CRIM'), (4, 'INDUS'), (5, 'ZN'), (6, 'TAX'), (7, 'B'), (8, 'AGE')]


## Linear Regression

In [9]:
np.random.seed(0)

x = np.random.normal(0, 1, (5000, 3))
y = x[:, 0] + 2*x[:, 1] + np.random.normal(0, 2, 5000)

lr = LinearRegression()
lr.fit(x, y)
lr.coef_

[[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 ..., 
 [ 1.91662273 -1.24813317  0.12762022]
 [ 1.20272166  0.62844209  1.83888091]
 [ 0.75309415 -0.58103281 -0.19837974]]
[ 7.50433081  7.15004266 -0.14733131 ..., -0.69416212  0.34174332
 -1.06202835]


array([ 0.98422873,  1.99522378, -0.04074316])

In [12]:
lr.intercept_

-0.057850509972938058

As can be seen in this example, the model certainly recovers the fundamental structure of the data very well, despite the significant noise we place in the data. __Repare how the coefficient of $x_2$ is practically double $ x_1 $, and how the noise coefficient ($ x_3 $ is practically zero) __. In fact, this problem is particularly simple for a linear model: __ purely linear relationship between attributes and variable response and no correlation between attributes.

When there are multiple (linearly) correlated attributes - as is the case with many real-world data - the model becomes unstable, ie small changes in data can cause large changes in model coefficients, making model interpretation very difficult , also called the "multicollinearity problem".

Let's see an example:

In [11]:
size = 100
np.random.seed(5)

x_seed = np.random.normal(0, 1, size)
x1 = x_seed + np.random.normal(0, 0.1, size)
x2 = x_seed + np.random.normal(0, 0.1, size)
x3 = x_seed + np.random.normal(0, 0.1, size)

y = x1 + x2 + x3 + np.random.normal(0, 1, size)
x = np.array([x1, x2, x3]).T

lr = LinearRegression()
lr.fit(x, y)
print(lr.coef_)

[-1.2912413   1.59097473  2.74727029]


__Regularization is a method to add restrictions or penalties to a model with the objective of preventing overfitting and improving generalization__. Instead of minimizing the cost function $ E (x, y) $, the cost function becomes $ E (x, y) + \ alpha \ | w \ | $, where $ w $ is the coefficient vector of the model, $ \ | \ cdot \ | $ is typically the norm L1 or L2 and $ \ alpha $ is a tunable parameter that specifies the force of regularization ($ \ alpha = 0 $ means "without regularization"). In relation to linear models, the two most used regularization methods are L1 and L2, also known as [Lasso](http://en.wikipedia.org/wiki/Least_squares#Lasso_method) and [Ridge](http://en.wikipedia.org/wiki/Tikhonov_regularization), respectively, when applied in linear regression.

### L1 regularization / Lasso

L1 regularization adds a penalty $\alpha \sum_{i=1}^n \left|w_i\right|$ to the loss function (L1-norm). Since each non-zero coefficient adds to the penalty, it forces weak features to have zero as coefficients. Thus L1 regularization produces sparse solutions, inherently performing feature selection.
For regression, Scikit-learn offers [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) for [Logistic regression ](http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)and Logistic regression with L1 penalty for classification.

In [13]:
boston = load_boston()
scaler = StandardScaler()
x = scaler.fit_transform(boston['data'])
y = boston['target']
names = boston['feature_names']

lasso = Lasso(alpha=0.3, random_state=42)
lasso.fit(x, y)

for c, att in zip(lasso.coef_, names):
    print('{}: {}'.format(att, round(c, 3)))

CRIM: -0.242
ZN: 0.082
INDUS: -0.0
CHAS: 0.54
NOX: -0.699
RM: 2.993
AGE: -0.0
DIS: -1.081
RAD: 0.0
TAX: -0.0
PTRATIO: -1.756
B: 0.628
LSTAT: -3.705


We see that a number of features have coefficient 0. If we increase α further, the solution would be sparser and sparser, i.e. more and more features would have 0 as coefficients.

Note however that L1 regularized regression is unstable in a similar way as unregularized linear models are, meaning that the coefficients (and thus feature ranks) can vary significantly even on small data changes when there are correlated features in the data. Which brings us to L2 regularization.

### L2 regularization / Ridge
L2 regularization (called ridge regression for linear regression) adds the [L2 norm](http://en.wikipedia.org/wiki/Norm_%28mathematics%29#Euclidean_norm)penalty ($\alpha\sum_{i=1}^n w_i^2$) to the loss function.

Since the coefficients are squared in the penalty expression, it has a different effect from L1-norm, namely it forces the coefficient values to be spread out more equally. For correlated features, it means that they tend to get similar coefficients. Coming back to the example of $y = x_1 + x_2$, with strongly correlated $X_1$ and $X_2$, then for L1, the penalty is the same whether the learned model is $y = 1*x_1 + 1*x_2$ or $y = 2*x_1 + 0*x_2$. In both cases the penalty is $2\alpha$. For L2 however, the first model’s penalty is $1^2 + 1^2 = 2\alpha$, while for the second model is penalized with $2^2 + 0^2 = 4\alpha$.

The effect of this is that models are much more stable (coefficients do not fluctuate on small data changes as is the case with unregularized or L1 models). So while L2 regularization does not perform feature selection the same way as L1 does, it is more useful for feature *interpretation*: a predictive feature will get a non-zero coefficient, which is often not the case with L1.

Lets look at the example with three correlated features again, running the example 10 times with different random seeds, to emphasize the stability of L2 regression.

In [14]:
size = 100

for i in range(10):
    print("Random seed:", i)
    np.random.seed(seed=i)
    x_seed = np.random.normal(0, 1, size)
    x1 = x_seed + np.random.normal(0, 0.1, size)
    x2 = x_seed + np.random.normal(0, 0.1, size)    
    x3 = x_seed + np.random.normal(0, 0.1, size)    
    y = x1 + x2 + x3 + np.random.normal(0, 1, size)
    x = np.array([x1, x2, x3]).T
    
    lr = LinearRegression()
    lr.fit(x, y)
    print("Linear model:", lr.coef_)
    
    ridge = Ridge(alpha=10)
    ridge = ridge.fit(x, y)
    print("Ridge model:", ridge.coef_)
    print()

Random seed: 0
Linear model: [ 0.7284403   2.30926001 -0.08219169]
Ridge model: [ 0.93832131  1.05887277  0.87652644]

Random seed: 1
Linear model: [ 1.15181561  2.36579916 -0.59900864]
Ridge model: [ 0.98409577  1.06792673  0.75855367]

Random seed: 2
Linear model: [ 0.69734749  0.32155864  2.08590886]
Ridge model: [ 0.97159124  0.94256202  1.08539406]

Random seed: 3
Linear model: [ 0.28735446  1.25386129  1.49054726]
Ridge model: [ 0.91891806  1.00474386  1.03276594]

Random seed: 4
Linear model: [ 0.18726691  0.77214206  2.1894915 ]
Ridge model: [ 0.96401621  0.98152524  1.0983599 ]

Random seed: 5
Linear model: [-1.2912413   1.59097473  2.74727029]
Ridge model: [ 0.75819864  1.01085804  1.1390417 ]

Random seed: 6
Linear model: [ 1.19909595 -0.0306915   1.91454912]
Ridge model: [ 1.01616507  0.89032238  1.0907386 ]

Random seed: 7
Linear model: [ 1.47386769  1.76236014 -0.15057274]
Ridge model: [ 1.0179376   1.03865514  0.90082373]

Random seed: 8
Linear model: [ 0.0840547   1.879

As you can see from the example, the coefficients can vary widely for linear regression, depending on the generated data. For L2 regularized model however, the coefficients are quite stable and closely reflect how the data was generated (all coefficients close to 1).
<hr>

# Practical example

To finalize this notebook, we will apply most of the attribute selection methods we have seen to compare them. The dataset we are going to use is called __Friedman \ # 1 regression dataset__ ([source](ftp://ftp.uic.edu/pub/depts/econ/hhstokes/e538/Friedman_mars_1991.pdf)). The data are generated according to the following formula:

$$y = 10sin(\pi x_1 x_2) + 20(x_3 – 0.5)^2 + 10x_4 + 5x_5 +\epsilon$$

where $x_1$ a $x_5$ are sampled from a uniform distribution and $\epsilon$ is a noise with normal distribution $N(0,1)$. In addition, the original dataset has five noise attributes $x_6, \dots, x_{10}$ independent of the response variable. We will increase the number of attributes with more 4 variables $x_{11}, \dots, x_{14}$ which are strongly correlated with $x_1,\dots,x_4$ respectively generated by:

$$f(x) = x + N(0, 0.01)$$

This produces a correlation coefficient greater than $0.999$ between the variables. Our objective with this example is to show how different ranking methods deal with correlations in data.

We will also normalize the ranking scores to be between 0 (for the least important attribute) and 1 (for the most important attribute). In the case of the recursive elimination of attributes, the 5 most important attributes will receive score 1, and the rest of the attributes will have the score normalized between 0 and 1 according to their position.

In [15]:
seed = 0
np.random.seed(seed)

size = 750
x = np.random.uniform(0, 1, (size, 14))
y = (10 * np.sin(np.pi*x[:,0]*x[:,1]) + 20*(x[:,2] - .5)**2 + 10*x[:,3] + 5*x[:,4] + np.random.normal(0,1))

# adiciona 3 variáveis correlacionados com x1-x3
x[:,10:] = x[:,:4] + np.random.normal(0, .025, (size,4))
 
names = ["x%s" % i for i in range(1,15)] 
ranks = {}

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = [round(x, 2) for x in ranks]
    return ranks

lr = LinearRegression(normalize=True)
lr.fit(x, y)
ranks["Lin. Reg."] = rank_to_dict(np.abs(lr.coef_), names)
 
ridge = Ridge(alpha=7)
ridge.fit(x, y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)
 
lasso = Lasso(alpha=.05)
lasso.fit(x, y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
 
rlasso = RandomizedLasso(alpha=0.04, random_state=seed)
rlasso.fit(x, y)
ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)
 
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(x, y)
ranks["RFE"] = rank_to_dict(list(map(float, rfe.ranking_)), names, order=-1)
 
f, pval  = f_regression(x, y, center=True)
ranks["Corr."] = rank_to_dict(f, names)
 
mine = MINE()
mic_scores = []
for i in range(x.shape[1]):
    mine.compute_score(x[:,i], y)
    m = mine.mic()
    mic_scores.append(m)

ranks["MIC"] = rank_to_dict(mic_scores, names)

df = pd.DataFrame.from_dict(ranks)
df = df.rename(index = dict(zip(range(14), names)))
df

,Lin. Reg.,Ridge,Lasso,Stability,RFE,Corr.,MIC
x1,1.00,0.77,0.79,0.77,1.00,0.30,0.39
x2,0.56,0.75,0.83,0.73,1.00,0.44,0.61
x3,0.50,0.05,0.00,0.00,1.00,0.00,0.34
x4,0.57,1.00,1.00,1.00,1.00,1.00,1.00
x5,0.27,0.88,0.51,0.64,0.78,0.10,0.20
x6,0.02,0.05,0.00,0.00,0.44,0.00,0.00
x7,0.00,0.01,0.00,0.00,0.00,0.01,0.07
x8,0.03,0.09,0.00,0.00,0.56,0.02,0.05
x9,0.00,0.00,0.00,0.00,0.11,0.01,0.09
x10,0.01,0.01,0.00,0.00,0.33,0.00,0.04


The results above have some interesting features:
- __Linear Regression__: As each attribute is evaluated independently, the scores for the attributes $x_1,\dots,x_4$ are very similar to $x_{11},\dots,x_14$, while the noise attributes $x_5,\dots,x10$ are correctly identified to have almost no relation to the response variable. It is not possible to identify any relationship between $x_3$ and $y$, since this relationship is quadratic (in fact, this applies to almost all methods except the MIC). It is also possible to observe that while the method is able to measure the linear relationship between each attribute and the response variable, it is not optimal to select the best attributes to improve the generalization of the model, since the most important attributes will be essentially chosen twice .
- __Lasso__: this method was able to select the best attributes, whereas other attributes to be close to zero. It is clearly useful when the intention is to reduce the number of attributes, but not necessarily to interpret the data (since it makes us believe that the attributes $x_{11},\dots, x_ {13}$ does not have a strong relation with the response variable).
- __MIC__: is similar to the correlation coefficient in relation to treating all attributes "equally". In addition, this method is able to find the nonlinear relationship between $x_3$ and the response variable.
- __Ridge Regression__: This method forces the regression coefficients to similarly spread among the correlated variables. This is clearly visible in the example where $x_{11},\dots,x_14$ are close to $x_1,\dots,x_4$ in terms of scores.
- __Stability Selection__: is generally able to make a good compromise between data interpretation and selection of better attributes for model improvement. This is well illustrated in the example. Like Lasso, this method is able to identify the best attributes ($x_1,x_2,x_4,x_5$). Likewise, the correlated variables $x_11,x12$ and $x14$ also have a high score, showing their relationship to the response.

## Summary
Attribute selection can be incredibly useful in a wide range of machine learning and data mining applications. __It is important to keep in mind why you are making attribute selection and understand which method works best for this purpose__. When selecting the best attributes to improve model performance, it is easy to verify that a particular method works well relative to other alternatives simply by performing cross-validation. However, it is not so simple when we use rankings to interpret the data, where the stability of the ranking method is crucial, and if the method does not have this property (like the Lasso regression), we can easily draw incorrect conclusions. What can be done is to sub-sample the data and run selection algorithms on the subsets. If the results are consistent on the subsets, it is relatively safe to rely on the stability of the method in that particular data and therefore easy to interpret the data in terms of rank.

## Referance
- [Feature Selection - Wikipedia](https://en.wikipedia.org/wiki/Feature_selection)
- [Selecting good features – Part I: univariate selection](http://blog.datadive.net/selecting-good-features-part-i-univariate-selection/)
- [Selecting good features – Part II: linear models and regularization](http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/)
- [Selecting good features – Part IV: stability selection, RFE and everything side by side](http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/)
- [Introduction to Feature Selection methods with an example (or how to select the right variables?)](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/)
- [BorutaPy](http://danielhomola.com/2015/05/08/borutapy-an-all-relevant-feature-selection-method/)